In [2]:
# Only for visualization. Irrelevant to results. 
def reverse_colourmap(cmap, name = 'my_cmap_r'):
    """
    In: 
    cmap, name 
    Out:
    my_cmap_r

    Explanation:
    t[0] goes from 0 to 1
    row i:   x  y0  y1 -> t[0] t[1] t[2]
                   /
                  /
    row i+1: x  y0  y1 -> t[n] t[1] t[2]

    so the inverse should do the same:
    row i+1: x  y1  y0 -> 1-t[0] t[2] t[1]
                   /
                  /
    row i:   x  y1  y0 -> 1-t[n] t[2] t[1]
    """        
    reverse = []
    k = []   

    for key in cmap._segmentdata:    
        k.append(key)
        channel = cmap._segmentdata[key]
        data = []

        for t in channel:                    
            data.append((1-t[0],t[2],t[1]))            
        reverse.append(sorted(data))    

    LinearL = dict(zip(k,reverse))
    my_cmap_r = mpl.colors.LinearSegmentedColormap(name, LinearL) 
    return my_cmap_r




#The purpose of this is to help plot hypocenter locations as if we are looking along strike of a fault.
def alongStrike(lon, lat, lonCenter, latCenter, strike):
    """returns: x, y . Both in meters and viewing perpendicular to fault.
    
    lon and lat are position of hypocenters to plot
    lonCenter and latCenter are the position of the one fault that other points are rotated around
    strike is the strike of the one fault that other points are rotated around
    
    """
    
    #First convert from lat lon to meters from center fault
    y, x = ChangeAxis(latCenter, lonCenter, lat, lon)
    
    #Now rotate the points so that we look along strike
    xR, yR = rotation( strike - (90)*np.pi/180, x, y)
    
    return xR, yR




# This is for visualization. It doesn't affect results. 
def set_size_ax(w,h, ax=None):
    """ w, h: width, height in inches. Doesn't return anything. 
    
    This is for visualization. It doesn't affect results. """
    if not ax: ax=plt.gca()
    l = ax.figure.subplotpars.left
    r = ax.figure.subplotpars.right
    t = ax.figure.subplotpars.top
    b = ax.figure.subplotpars.bottom
    figw = float(w)/(r-l)
    figh = float(h)/(t-b)
    ax.figure.set_size_inches(figw, figh)

    
    
    
# # Assigns colors (as list) to points based on whether none, 1, or both focal planes has positive dcs
# def greenRed(dCS,dCS2, high = 0.005e6, low = -0.005e6, printProp = False):
#     """returns (color, legend_markers, labels) for plots
    
#     # dCS and dCS2 are the dCS on different planes
#     # high and low: If both values are between high and low, grey is assigned
#     # printProp: show proportion of positive to negative. Leave as False for now."""
    
#     # Make booleans to show which color range the hypocenter will be assigned
#     big1 = dCS>high
#     big2 = dCS2>high
#     small1 = dCS<low
#     small2 = dCS2<low
#     neut1 = (~big1)*(~small1)
#     neut2 = (~big2)*(~small2)
    
#     # Append green if both planes were > high
#     # Append blue if just one plane was < high
#     # Append red if both planes were < low
#     # Append grey if both planes were between high and low.
#     color = []
#     for i in range(dCS.size):
#         if  (big1[i]*big2[i]).astype('bool'):
#             color.append('red')
#         elif(big1[i]+big2[i]).astype('bool'):
#             color.append('green')
#         elif(small1[i]*small2[i]).astype('bool'):
#             color.append('blue')
#         else:
#             color.append('grey')

#     # Just the legend points.
#     marker1 = plt.scatter([],[], s=10, c = 'red')
#     marker2 = plt.scatter([],[], s=10, c = 'green')
#     marker3 = plt.scatter([],[], s=10, c = 'blue')
#     marker4 = plt.scatter([],[], s=10, c = 'grey')
#     legend_markers = [marker1, marker2, marker3, marker4]

#     # Labels for legend
#     labels = [
#         '$\Delta$CFF'+' > '+str(round(high*1e-6,3))+' MPa for both planes',
#         '$\Delta$CFF'+' > '+str(round(high*1e-6,3))+' MPa for one plane',
#         '$\Delta$CFF'+' < '+str(round(-low*1e-6,3))+' MPa for both planes',
#         str(round(low*1e-6,3))+' < '+'$\Delta$CFF'+' < '+str(round(high*1e-6,3))+' MPa for both planes',
#         ]

#     # Leave be for now
#     if printProp:
#         print(np.sum((big1+big2).astype('bool'))/np.sum(small1))

#     return color, legend_markers, labels

def greenRed(dCS,dCS2, high = 0.005e6, low = -0.005e6, printProp = False):
    """returns (color, legend_markers, labels) for plots
    
    # dCS and dCS2 are the dCS on different planes
    # high and low: If both values are between high and low, grey is assigned
    # printProp: show proportion of positive to negative. Leave as False for now."""
    
    # Make booleans to show which color range the hypocenter will be assigned
    big1 = dCS>high
    big2 = dCS2>high
    small1 = dCS<low
    small2 = dCS2<low
    neut1 = (~big1)*(~small1)
    neut2 = (~big2)*(~small2)
    
    # Append green if both planes were > high
    # Append blue if just one plane was < high
    # Append red if both planes were < low
    # Append grey if both planes were between high and low.
    color = []
    for i in range(dCS.size):
        if  (big1[i]*big2[i]).astype('bool'):
            color.append('red')
        elif(big1[i]+big2[i]).astype('bool'):
            color.append('yellow')
        elif(small1[i]*small2[i]).astype('bool'):
            color.append('blue')
        else:
            color.append('grey')

    # Just the legend points.
    marker1 = plt.scatter([],[], s=10, c = 'red')
    marker2 = plt.scatter([],[], s=10, c = 'green')
    marker3 = plt.scatter([],[], s=10, c = 'blue')
    marker4 = plt.scatter([],[], s=10, c = 'grey')
    legend_markers = [marker1, marker2, marker3, marker4]

    # Labels for legend
    labels = [
        '$\Delta$CFF'+' > '+str(round(high*1e-6,3))+' MPa for both planes',
        '$\Delta$CFF'+' > '+str(round(high*1e-6,3))+' MPa for one plane',
        '$\Delta$CFF'+' < '+str(round(-low*1e-6,3))+' MPa for both planes',
        str(round(low*1e-6,3))+' < '+'$\Delta$CFF'+' < '+str(round(high*1e-6,3))+' MPa for both planes',
        ]

    # Leave be for now
    if printProp:
        print(np.sum((big1+big2).astype('bool'))/np.sum(small1))

    return color, legend_markers, labels



# Plot the slip distribution on a fault
def plotSlip(dx, dy):
    """plots the slip distribution for an inhomogenous fault
    
    dx and dy are the slip in x and y. 
    provide dx and dy as 2d arrays."""
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(121, aspect='equal')
    mag = np.sqrt(dx**2+dy**2)
    
    # colors represent magnitude of slip
    plt.imshow(mag.T, origin = 'lower')
    plt.colorbar()
    x, y = np.meshgrid( np.arange(dx.shape[1]), np.arange(dx.shape[0]), indexing = 'ij' )
    
    # add vectors using quiver
    ax.quiver(x.T, y.T, dx.T, dy.T, color = 'b', units='x', scale = 1, angles = 'xy')
    plt.show()

    
    
    
# Used to plot dx, dy, and dz at GPS stations during the fault inversion process
def plotDxyz(ax, lat, lon, dxO, dxM, dyO, dyM):
    """plot vectors dx and dy (observed and modeled). 
    
    lat and lon are position of vector tails.
    dx and dy are x and y displacements.
    O or M signifies observed and modelled values."""
    scalePlot = 1/10

    
    edgecolor='black'
    linewidth=.5
    ax.set_xlim(13.1, 13.8)
    ax.set_ylim(42.1, 42.6)
    ax.quiver(lon, lat, dxO, dyO, color = 'r', units='inches', scale = scalePlot, 
              angles = 'uv', edgecolor=edgecolor, linewidth=linewidth)
    ax.quiver(lon, lat, dxM, dyM, color = 'b', units='inches', scale = scalePlot, 
              angles = 'uv', edgecolor=edgecolor, linewidth=linewidth)
    
    # A scatter plot to show vector tail locations
    ax.scatter(lon, lat, c='k', s=5)
    
    
    
# The purpose of this is to plot hypocenters while viewing perpendicular to strike.
def plotAgainstStrike(minML, plots, mL, lonFault, latFault, depthFault, lonFaultA, latFaultA, depthFaultA,dCS = False):
    
    laqPos = mL==np.amax(mL);lonLaq = lonFault[laqPos]; latLaq = latFault[laqPos]
    xPerpendicular, yParallel = alongStrike(lonFault, latFault, lonLaq, latLaq, 130*np.pi/180)
    xPerpendicularA, yParallelA = alongStrike(lonFaultA, latFaultA, lonLaq, latLaq, 130*np.pi/180)

    dY = (np.amax(yParallel) - np.amin(yParallel)) / plots
    locations = np.arange(plots)*dY+np.amin(yParallel)
    for i in np.arange(plots-1):
        fig, ax = plt.subplots()
        yParallelMax = locations[i+1]
        yParallelMin = locations[i]
        boo = (yParallel>yParallelMin)*(yParallel<yParallelMax)
        booA = (yParallelA>yParallelMin)*(yParallelA<yParallelMax)
        booM = boo * mL>minML
        ax.scatter(xPerpendicularA[booA],-depthFaultA[booA],s=2, c ='violet')
        
        if dCS.any():
            color, legend_markers, labels = greenRed(dCS[boo], dCS2[boo])
            s = ax.scatter(xPerpendicular[boo],-depthFault[boo],s=20, c =  color, linewidths = .3, edgecolors = 'k')
            
        ax.scatter(xPerpendicular[booM],-depthFault[booM],s=400, c = 'r', marker = '*',
                   vmin = -2, vmax = 2, linewidths = .3, edgecolors = 'k')
        
        index = np.arange(mL.size)
        indexMag = index[booM]
        for a in indexMag:
            ax.annotate(a, (xPerpendicular[a],-depthFault[a]), color = 'k')
        plt.xlim(-10000, 20000)
        plt.ylim(-30000, 0)
        set_size_ax(10,10)
        plt.show()

In [ ]:
# Used for comparing stress to distance from fault cells.


# This is a series of functions for finding the distance from a point 
# to some part of a fault

#distance to the edges of a fault. 
def distEdge(xP, yP, sLength, dLength):
    """Assumes strike is 0 and fault center is 0,0 """
    side1 =  .5 * sLength
    side2 = -.5 * sLength
    dist1 = np.sqrt( (side1-yP)**2 + (xP)**2 )
    dist2 = np.sqrt( (side2-yP)**2 + (xP)**2 )
    distMin = np.zeros(dist1.shape)
    distMin[dist1<dist2]=dist1[dist1<dist2]
    distMin[dist1>dist2]=dist2[dist1>dist2]
    return dist1, dist2, distMin


# Distance to whichever cell has the nearest center. 
def distNearestGrid(xCents, yCents, xG, yG):
    """returns distMin.
    
    distMin gives the distance from point xG,yG to the nearest xCents, yCents point.
    """    
    xCents = xCents.reshape((1, xCents.size))
    yCents = yCents.reshape((1, xCents.size))
    xG = xG.reshape((1, xG.size))
    yG = yG.reshape((1, xG.size))
    
    # dists[Cent, G] gives distance from Cent point to G point
    dists = np.sqrt( (xCents.T-xG)**2 + (yCents.T-yG)**2 )
    
    distMin = np.zeros(xG.size)
    for i in np.arange(xG.size):
        distMin[i] = np.amin(dists[:, i])

    return distMin


# Generate a meshed grid to test stress functions on
def testGrid(xmin, xmax, ymin, ymax, zmin, zmax,
             nx = 50, ny = 50, nz = 1): # dCS grid and parameters

    # 2d Mesh values. Can be made 3d by changing zz and nz
    xx = np.linspace(xmin,xmax,nx)
    yy = np.linspace(ymin,ymax,ny)
    zz = np.linspace(zmin,zmax,nz)
    xGrid, yGrid, zGrid = np.meshgrid(xx,yy,zz)
    totalPoints=nx*ny*nz
    lonGrid=xGrid.reshape(totalPoints)
    latGrid=yGrid.reshape(totalPoints)
    depthGrid=zGrid.reshape(totalPoints)
    
    return lonGrid, latGrid, depthGrid

In [ ]:
# for plotting 3d surface on plotly
def tri_indices(simplices):
    #simplices is a numpy array defining the simplices of the triangularization
    #returns the lists of indices i, j, k

    return ([triplet[c] for triplet in simplices] for c in range(3))

def map_z2color(zval, colormap, vmin, vmax):
    #map the normalized value zval to a corresponding color in the colormap

    if vmin>vmax:
        raise ValueError('incorrect relation between vmin and vmax')
    t=(zval-vmin)/float((vmax-vmin))#normalize val
    R, G, B, alpha=colormap(t)
    return 'rgb('+'{:d}'.format(int(R*255+0.5))+','+'{:d}'.format(int(G*255+0.5))+\
           ','+'{:d}'.format(int(B*255+0.5))+')'

def plotly_trisurf(x, y, z, simplices, colormap=cm.RdBu, plot_edges=None):
    #x, y, z are lists of coordinates of the triangle vertices 
    #simplices are the simplices that define the triangularization;
    #simplices  is a numpy array of shape (no_triangles, 3)
    #insert here the  type check for input data

    points3D=np.vstack((x,y,z)).T
    tri_vertices=map(lambda index: points3D[index], simplices)# vertices of the surface triangles     
    zmean=[np.mean(tri[:,2]) for tri in tri_vertices ]# mean values of z-coordinates of 
                                                      #triangle vertices
    min_zmean=np.min(zmean)
    max_zmean=np.max(zmean)
    facecolor=[map_z2color(zz,  colormap, min_zmean, max_zmean) for zz in zmean]
    I,J,K=tri_indices(simplices)

    triangles=go.Mesh3d(x=x,
                     y=y,
                     z=z,
#                      facecolor=facecolor,
                     opacity=.7,
                     i=I,
                     j=J,
                     k=K
                    )

    if plot_edges is None:# the triangle sides are not plotted 
        return [triangles]
    else:
        #define the lists Xe, Ye, Ze, of x, y, resp z coordinates of edge end points for each triangle
        #None separates data corresponding to two consecutive triangles
        lists_coord=[[[T[k%3][c] for k in range(4)]+[ None]   for T in tri_vertices]  for c in range(3)]
        Xe, Ye, Ze=[reduce(lambda x,y: x+y, lists_coord[k]) for k in range(3)]

        #define the lines to be plotted
        lines=go.Scatter3d(x=Xe,
                        y=Ye,
                        z=Ze,
                        mode='lines',
                        line=dict(color= 'rgb(50,50,50)', width=1.5)
               )
        return [triangles, lines]

# Plot stress results

In [1]:
# find dCS for all 3 planes and on a grid for each mainshock model. 

def dCS_each_plane(latF, lonF, depthF,
                   sF, dF, rF,
                   dipSlip, strikeSlip, tensile, 
                   dipLength, strikeLength,
                   
                   constants):
    """ returns (CS1, CS2, CSM, CSG) 
    Return values are dCS for plane 1, 2, mainshock plane, and on the grid,
        each time using the same source fault. 
    
    constants are: 
        (latP, lonP, depthP,
        latG, lonG, depthG,

        s1, d1, r1,
        s2, d2, r2,
        sG, dG, rG,

        ttF, ttP
        shearMod, lambdaLame, coefFrict, coefSkempt)
        
    Constant names are basically the same as in sumAll.
    
        
    """
    # Take out the constants (these are a tupple to save space when calling function)
    (latP, lonP, depthP,
    latG, lonG, depthG,

    s1, d1, r1,
    s2, d2, r2,
    sG, dG, rG,

    ttF, ttP,
    shearMod, lambdaLame, coefFrict, coefSkempt
    )=constants
    
    # plane 1 dCS
    CS1 = sumAll(shearMod, lambdaLame, coefFrict, coefSkempt,  

               latF, lonF, depthF, 
               latP, lonP, depthP, 

               dipLength, strikeLength, 
               dipSlip, strikeSlip, tensile,

               sF, dF, rF,
               s1, d1, r1,

               ttF, ttP)[0]
    
    # plane 2 dCS
    CS2 = sumAll(shearMod, lambdaLame, coefFrict, coefSkempt,  

               latF, lonF, depthF, 
               latP, lonP, depthP, 

               dipLength, strikeLength, 
               dipSlip, strikeSlip, tensile,

               sF, dF, rF,
               s2, d2, r2,

               ttF, ttP)[0]  
    
    # mainshock plane dCS
    sM = np.average(sG) * np.ones(latP.size)
    dM = np.average(dG) * np.ones(latP.size)
    rM = np.average(rG) * np.ones(latP.size)
    
    CSM = sumAll(shearMod, lambdaLame, coefFrict, coefSkempt,  

           latF, lonF, depthF, 
           latP, lonP, depthP, 

           dipLength, strikeLength, 
           dipSlip, strikeSlip, tensile,

           sF, dF, rF,
           sM, dM, rM,

           ttF, ttP)[0]
    
    # grid dCS
    sG = np.average(sG) * np.ones(latG.size)
    dG = np.average(dG) * np.ones(latG.size)
    rG = np.average(rG) * np.ones(latG.size)
    
    CSG = sumAll(shearMod, lambdaLame, coefFrict, coefSkempt,  

               latF, lonF, depthF, 
               latG, lonG, depthG, 

               dipLength, strikeLength, 
               dipSlip, strikeSlip, tensile,

               sF, dF, rF,
               sG, dG, rG)[0]

    return CS1, CS2, CSM, CSG





# provide quick access to dCS_each_plane
def run_dCS_each_plane(model, constInfo, fName = None,
              load=True, rerun=False, save=False,):
    # dCS from INGV inverted mainshock
    if (load or save) and fName is None:
        print('provide fName')

    if rerun:
        dcsmodel = dCS_each_plane(model.latL, model.lonL, model.depthL,
                      model.sL, model.dL, model.rL,
                      model.dsL, model.ssL, model.tenL, 
                      model.dipLengthL, model.strikeLengthL,
                  constInfo)

    if save:
        saveArrays(dcsmodel, fName)

    if load:
        dcsmodel = loadArrays(fName)

    dCS1, dCS2, dCSM, dCSG = dcsmodel 
    dCSG = dCSG.reshape(nx, ny)
    return dCS1, dCS2, dCSM, dCSG



# Used in plotting the results of dCS
def dCS_sca_im(ax, high, low, lat, lon, dCS1, dCS2=None,
                 dCSG = None, strainers=None, maxIm = 0.5, minIm = -0.5):
            
    if dCSG is not None:
        im = ax.imshow(dCSG*1e-6, extent = (xmin, xmax, ymin, ymax), 
               origin = 'lower',interpolation = 'bilinear', zorder=0,
               cmap = 'seismic', vmax = maxIm, vmin = minIm)
        plt.colorbar(im, ax=ax, label = 'MPa')

    if dCS2 is None:
        dCS2 = dCS1.copy()
    
    color, legend_markers, labels = greenRed(dCS1, dCS2, high, low)
    
    scatterP = ax.scatter(lon, lat, c = color, s = 10, zorder=1,
        linewidths=.5,edgecolors='black' )
    
    if strainers is not None:
        scatterF = ax.scatter(lon[strainers], lat[strainers], c = color, s = 500, 
            marker = '*', zorder = 2)

    ax.legend(handles=legend_markers, labels=labels, loc='upper right',
         scatterpoints=1)

    return im #, scatterP, scatterF

In [ ]:
# function to find if 0, 1, or 2 planes have positive cff, used in Figure 10
def p012(stress1, stress2, inclusion = None, 
         weight = None, weights = None, 
         asPercent = True):
    """
    Give stress on each plane as arrays.
    Returns p0, p1, p2 which are number of events with positive stress
        on 0, 1, or 2 planes. 
    Inclusion can be bool to decide which events to use. 
    """
    
    points = stress1.size
    if inclusion is None:
        inclusion = np.ones(points, dtype = bool)
        
    if weights is None:
        weights = np.ones(points)
    weightsSub = weights[inclusion]

    pos = (stress1[inclusion]>0).astype(int) + (stress2[inclusion]>0).astype(int)
    p0 = np.sum( (pos==0).astype(int) *weightsSub)
    p1 = np.sum( (pos==1).astype(int) *weightsSub)
    p2 = np.sum( (pos==2).astype(int) *weightsSub)

    if asPercent:
        p0 *= 100/np.sum(inclusion*weights)
        p1 *= 100/np.sum(inclusion*weights)
        p2 *= 100/np.sum(inclusion*weights)
        
    return p0, p1, p2